## Import airport_df

In [1]:
import pandas as pd
import sqlalchemy
sys.path.append('..\keys')
import key

schema="gans_db"   # name of the database you want to use here
host="localhost"        # to connect to your local server
user="root"
password=key.SQL_PASSWORD # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
airport_df = pd.read_sql_table('airport',con=con)

In [2]:
airport_df

,city_id,icao
0,1,LTBA
1,3,EGLC
2,4,RU-0001
3,4,ULLI
4,5,EDDB
5,6,LECU
6,6,LEGT
7,6,LEMD
8,7,UKKK
9,8,LIRA


## Connect to API

In [4]:
RERUN_TIME_IN_HOURS = 24
FLIGHT_FORCAST_TIMEWINDOW = 12

In [5]:
import math
from pytz import timezone
from datetime import datetime, date, timedelta

today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
rerun_window = math.ceil(RERUN_TIME_IN_HOURS/24) #roundup to bigger integer
next_run = (today + timedelta(days=rerun_window))
next_run

datetime.date(2023, 1, 6)

In [7]:
def connect_flight_api(icao_l):
    import requests 
    times = [["00:00","11:59"],["12:00","23:59"]]
    responses =[]
    for icoa in icao_l:
        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_id}/{next_run}T{time[0]}/{next_run}T{time[1]}"
            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
            headers = {
				"X-RapidAPI-Key": f"{key.Flight_API_key}",
				"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
			}
            responses.append(requests.request("GET", url, headers=headers, params=querystring))
    return responses

In [ ]:
def nextday_arrival_flights(icao_l):
    import json
    responses = connect_flight_api(icao_l)
    for res in responses:
        if res.status_code != 200:
            print(f"Negative Response,{res.text}")
        else:
            flights=res.json()
            print(json.dumps(flights))
            print("=========================")
            for flight in flights["arrivals"]:
                arrival_time_local = flight["arrival"].get('scheduledTimeLocal', "unknown")
                arrival_terminal = flight['arrival'].get('terminal', "unknown")
                departure_city =  flight["departure"]["airport"].get("name", "unknown")
                departure_icao =  flight["departure"]["airport"].get("icao", "unknown")
                departure_time_local = flight["departure"].get("scheduledTimeLocal", "unknown")
                airline  =  flight["airline"].get("name", "unknown")
                flight_number  =  flight.get("number", "unknown")
                data_retrieved_on  =  datetime.now().astimezone(timezone('Europe/Berlin')).date()


In [ ]:
	#TODO: Replace with list
 airport_id_l =["EGLC"]
nextday_arrival_flights(airport_id_l)

In [27]:
json.dumps(responses[0].json())

In [22]:
arrival_time_local

'2023-01-05 07:55+00:00'